In [ ]:
# Global variables
import torch

gpu = torch.cuda.is_available()
device = "cuda" if gpu else "cpu"
tdevice = torch.device(device)

# Loading the data with some normalisation
As the grid is fixed, we normalise X, Y, Z, T

T might be changed.

In [ ]:
# from inr_src import return_dataset
import inr_src as inr
path = "./data/test_data.npy"
ds, ds_test, nv, nv_y = inr.return_dataset(path, normalise_targets=False, gpu=False)

print("##########\n X, Y, T")
print(ds.samples)
print("##########\n Z")
print(ds.targets)
print("##########\n Shape")
print(ds.targets.shape)

# Implicit Neural Representation

Currently, different models are available: RFF or SIREN. I could also implement WIRES (based on wavelet functions).
For these models, different archectures are available: 

The loss we are minimizing is the: $$\mathcal{L} = ||f(x, y, t)  - z ||_2 + \lambda_1 ||f(x, y, t)  - z ||_1 + \lambda_{xy}||\frac{\mathrm{d} f}{\mathrm{d} xy}||_2 + \lambda_{t}||\frac{\mathrm{d} f}{ \mathrm{d} t}||_2 $$ 

In [ ]:
# Some libraries
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

from scipy import interpolate
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [ ]:
path = "./data/test_data.npy"

opt = inr.AttrDict()
opt.path = path
opt.gpu = gpu
opt.model_name = "wires" # or siren or wires or RFF
opt.name = "wires_notebook_unnormalised"
opt.normalise_targets = False
model_hp = inr.AttrDict()

opt.fourier = opt.model_name == "RFF"
opt.siren = opt.model_name == "siren"
opt.wires = opt.model_name == "wires"
model_hp.fourier = opt.model_name == "RFF"
model_hp.siren = opt.model_name == "siren"
model_hp.wires = opt.model_name == "wires"


model_hp.verbose = True
model_hp.epochs = 10

model_hp.bs = 2**16
model_hp.scale = 5
model_hp.lr = 1e-3
model_hp.output_size = 1

if opt.siren or opt.wires:
    print(f"Using {opt.model_name}")
    model_hp.architecture = opt.model_name
    model_hp.hidden_num = 3
    model_hp.hidden_dim = 256
    model_hp.do_skip = True
    if opt.wires:
        model_hp.width_gaussian = 10.0
else:
    model_hp.mapping_size = 512
    model_hp.architecture = "skip-5"  # "Vlarge"
    model_hp.activation = "tanh"

model_hp.lambda_l1 = 0.0
model_hp.lambda_t = 0.0
model_hp.lambda_xy = 0.0

model, model_hp = inr.train(opt, model_hp, gpu=opt.gpu)


In [ ]:
test

In [ ]:
# Or if you prefer to load the model
opt = inr.AttrDict()
opt.name = "wires_notebook_unnormalised__test546"
## From saved
npz = np.load(f"meta/{opt.name}.npz")


weights = f"meta/{opt.name}.pth"

model_hp = inr.AttrDict(npz)

# model_hp.hidden_dim = model_hp.siren_hidden_dim
# model_hp.hidden_num = model_hp.siren_hidden_num
# model_hp.do_skip = model_hp.siren_skip
model_hp.normalise_targets = False
model_hp = inr.util_train.clean_hp(model_hp)


model = inr.ReturnModel(
    model_hp.input_size,
    output_size=model_hp.output_size,
    arch=model_hp.architecture,
    args=model_hp,
)
print(f"loading weight: {weights}")
model.load_state_dict(torch.load(weights, map_location=tdevice))

# Some data visualisation
## Some real data

In [ ]:
# Visualisation the data set by tempoarl averaging of each month
pc = np.load("data/test_data.npy")
q33 = np.quantile(pc[:,3], 0.33)
q66 = np.quantile(pc[:,3], 0.66)
pc_t0 = pc[pc[:,3] < q33]
pc_t1 = pc[(pc[:,3] >= q33) & (pc[:,3] < q66)]
pc_t2 = pc[(pc[:,3] >= q66)]

for spc, title in [(pc_t0, "Jan."), (pc_t1, "Feb."), (pc_t2, "Mar.")]:

    idx = np.random.choice(np.arange(spc.shape[0]), replace=False, size=int(1e5))
    fig = px.scatter_3d(x=spc[idx,0], y=spc[idx,1], z=spc[idx,2],
                color=spc[idx,3], width=600, height=600)
    fig.update_layout(title=title, legend_title_text="Time")
    fig.update_traces(marker_size=1)
    fig.show()


In [ ]:
spc = pc_t0
idx = np.random.choice(np.arange(spc.shape[0]), replace=False, size=int(1e5))
fig = px.scatter(x=spc[idx,0], y=spc[idx,1], color=spc[idx,2], width=600, height=600, color_continuous_scale=px.colors.sequential.Viridis)
            #color=spc[idx,3], width=1000, height=1000)
fig.update_layout(
    title=dict(text="January", font=dict(size=50), automargin=True, yref='paper'),
    font_size=40,
    xaxis_title="Latitude",
    yaxis_title="Longitude",
    legend_title="Legend Title",
    coloraxis_colorbar=dict(
        title="Height (m)",
    ),
)
fig.update_traces(marker_size=3)
fig.show()

In [ ]:
fig = px.scatter(x=spc[idx,0], y=spc[idx,1], color=spc[idx,3], 
                 width=600, height=600, 
                 color_continuous_scale=[(0, "red"), (0.5, "green"), (1, "blue")], )

fig.update_layout(
    title=dict(text="", font=dict(size=50), automargin=True, yref='paper'),
    font_size=40,
    xaxis_title="Latitude",
    yaxis_title="Longitude",
    legend_title="Legend Title",
    coloraxis_colorbar=dict(
        title="",
    ),
)
fig.update_coloraxes(colorbar_tickvals=[np.quantile(spc[idx,3], perc) for perc in [0.04,0.2,0.4,0.6,0.8,0.96]], colorbar_ticktext=[r't<sub>{}</sub>'.format(i) for i in range(6)],)
fig.update_traces(marker_size=3)
fig.show()

In [ ]:

def interpolate_missing_pixels(
        image: np.ndarray,
        mask: np.ndarray,
        method: str = 'nearest',
        fill_value: int = 0
):
    """
    :param image: a 2D image
    :param mask: a 2D boolean image, True indicates missing values
    :param method: interpolation method, one of
        'nearest', 'linear', 'cubic'.
    :param fill_value: which value to use for filling up data outside the
        convex hull of known pixel values.
        Default is 0, Has no effect for 'nearest'.
    :return: the image with missing values interpolated
    """
    h, w = image.shape[:2]
    xx, yy = np.meshgrid(np.arange(w), np.arange(h))

    known_x = xx[~mask]
    known_y = yy[~mask]
    known_v = image[~mask]
    missing_x = xx[mask]
    missing_y = yy[mask]

    interp_values = interpolate.griddata(
        (known_x, known_y), known_v, (missing_x, missing_y),
        method=method, fill_value=fill_value
    )

    interp_image = image.copy()
    interp_image[missing_y, missing_x] = interp_values

    return interp_image

df = pd.DataFrame(spc[idx])
df.columns = ["x", "y", "z", "Time"]
df['ybin'] = pd.cut(df.y, [t for t in np.arange(df.y.min(), df.y.max()+0.1, 0.1)])
df['xbin'] = pd.cut(df.x, [t for t in np.arange(df.x.min(), df.x.max()+0.01, 0.01)])
pvtdf = df.pivot_table(index='xbin', columns=['ybin'], values='z',
                    aggfunc=('mean'))
z = pvtdf.values
mask = np.ma.masked_invalid(z)
test = interpolate_missing_pixels(z, mask.mask, method='linear')


fig = go.Figure()
fig.add_trace(go.Contour(x=spc[idx,0],y=spc[idx,1],z=spc[idx,2],line_smoothing=1.3))
fig.update_layout(autosize=False)
fig.show()

## Trained model predicts Z from XYT of the trained points! (Should be good...)

In [ ]:
idx = 50
print(model(xytz_ds.samples[idx, :]))
print(xytz_ds.samples[idx, :])

In [ ]:
model

In [ ]:
model_hp

In [ ]:
xytz_ds = inr.XYTZ(
        path,
        train_fold=False,
        train_fraction=0.0,
        seed=42,
        pred_type="pc",
        nv=tuple(model_hp.nv),
        nv_targets=tuple(model_hp.nv_target),
        normalise_targets=model_hp.normalise_targets,
        gpu=gpu
    )

prediction = inr.predict_loop(xytz_ds, 2048, model, device=device)

mse_norm = mean_squared_error(xytz_ds.targets, prediction)
mae_norm = mean_absolute_error(xytz_ds.targets, prediction)

model_hp.nv_target = np.array(model_hp.nv_target)
model_hp.nv = np.array(model_hp.nv)

norm_target = xytz_ds.targets * model_hp.nv_target[0,1] + model_hp.nv_target[0,0]
norm_pred = prediction * model_hp.nv_target[0,1] + model_hp.nv_target[0,0]
mse_unnorm = mean_squared_error(norm_target, norm_pred)
mae_unnorm = mean_absolute_error(norm_target, norm_pred)

print(f"Normalised data -> MSE: {mse_norm:.5f} MAE: {mae_norm:.5f}")
print(f"True Z values ->   MSE: {mse_unnorm:.3f} MAE: {mae_unnorm:.3f}")

q33 = np.quantile(xytz_ds.samples[:,2], 0.33)
q66 = np.quantile(xytz_ds.samples[:,2], 0.66)

idx_0 = xytz_ds.samples[:,2] < q33
idx_1 = (xytz_ds.samples[:,2] >= q33) & (xytz_ds.samples[:,2] < q66)
idx_2 = (xytz_ds.samples[:,2] >= q66)

for idx, title in [(idx_0, "Jan."), (idx_1, "Feb."), (idx_2, "Mar.")]:
    samples = xytz_ds.samples[idx] * model_hp.nv[:,1] + model_hp.nv[:,0]
    pred = prediction[idx,0] * model_hp.nv_target[0,1] + model_hp.nv_target[0,0]

    idx = np.random.choice(np.arange(samples.shape[0]), replace=False, size=int(1e5))
    fig = px.scatter_3d(x=samples[idx,0], y=samples[idx,1], z=pred[idx],
                color=samples[idx,2])
    fig.update_layout(title=title, legend_title_text="Time")
    fig.update_traces(marker_size=2)
    fig.show()


# Predicting along the grid with "t = middle of the month"

In [ ]:
# Compute linear grid for plotting
#Capture a month
q16 = np.quantile(pc[:,3], 0.1666)
q50 = np.quantile(pc[:,3], 0.50)
q83 = np.quantile(pc[:,3], 0.8333)

dataset16 = inr.XYTZ_grid(pc, q16, nv=model_hp.nv, step_grid=0.01, gpu=gpu)
dataset50 = inr.XYTZ_grid(pc, q50, nv=model_hp.nv, step_grid=0.01, gpu=gpu)
dataset83 = inr.XYTZ_grid(pc, q83, nv=model_hp.nv, step_grid=0.01, gpu=gpu)
nv = np.array(model_hp.nv)
nv_target = np.array(model_hp.nv_target)
XYT16 = dataset16.samples * nv[:,1] + nv[:,0]
device = "cuda" if gpu else "cpu"
# Predict along the grid
preds16 = inr.predict_loop(dataset16, 2048, model, device=device)
preds50 = inr.predict_loop(dataset50, 2048, model, device=device)
preds83 = inr.predict_loop(dataset83, 2048, model, device=device)

for spred, title in [(preds16, "Jan."), (preds50, "Feb."), (preds83, "Mar.")]:
    spred_z = spred * nv_target[0,1] + nv_target[0,0]
    fig = px.scatter_3d(x=XYT16[:,0], y=XYT16[:,1], z=spred_z[:,0],
                color=spred_z[:,0])
    fig.update_layout(title=title, legend_title_text="Time")
    fig.update_traces(marker_size=2)
    fig.show()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Contour(x=XYT16[:,0],y=XYT16[:,1],z=spred_z[:,0],line_smoothing=1.3, colorscale=px.colors.sequential.Viridis))
fig.update_layout(width=600, height=1000)
fig.show()

In [ ]:
# creating animated gif for error
from data_src.plot_utils import f, plot_scatter

xytz_ds = inr.XYTZ(
        path,
        train_fold=False,
        train_fraction=0.0,
        seed=42,
        pred_type="pc",
        nv=model_hp.nv,
        nv_targets=model_hp.nv_target,
        gpu=gpu
    )

prediction_ds = inr.predict_loop(xytz_ds, 2048, model, device=device)
prediction_ds = prediction_ds * model_hp.nv_target[0,1] + model_hp.nv_target[0,0]
true_target = xytz_ds.targets * model_hp.nv_target[0,1] + model_hp.nv_target[0,0]
mae_error = true_target - prediction_ds
mse_error = np.sign(mae_error) * (mae_error)**2



sub_samples, sub_target = xytz_ds.samples, xytz_ds.targets
sub_samples = sub_samples * model_hp.nv[:,1] + model_hp.nv[:,0]
sub_target = sub_target * model_hp.nv_target[0,1] + model_hp.nv_target[0,0]
sub_pred = prediction_ds[:,0] * model_hp.nv_target[0,1] + model_hp.nv_target[0,0]


xrange = [sub_samples[:,0].min(), sub_samples[:,0].max()]
yrange = [sub_samples[:,1].min(), sub_samples[:,1].max()]

unique_times = np.unique(sub_samples[:,-1].numpy())
unique_times.sort()

dataset_t = inr.XYTZ_grid(pc, 0, nv=model_hp.nv, step_grid=0.01, gpu=gpu)
XYT_xy = dataset_t.samples * model_hp.nv[:,1] + model_hp.nv[:,0]

folder = "gif/wires"

for t in unique_times:#
    dataset_t.samples[:,-1] = (t - model_hp.nv[-1,0]) / model_hp.nv[-1,1]
    idx_t = np.where(sub_samples[:,-1] == t)[0]
    sub_sample_t = sub_samples[idx_t]
    sub_target_t = sub_target[idx_t]

    prediction = inr.predict_loop(dataset_t, 2048, model, device=device)
    prediction = prediction * model_hp.nv_target[0,1] + model_hp.nv_target[0,0]


    plot_scatter(XYT_xy, prediction.numpy()[:,0], [100, 1700], t, f"{folder}/surface/{f(t)}.png", px.colors.sequential.Viridis, xrange=xrange, yrange=yrange, isoline=True)


    plot_scatter(sub_sample_t, sub_target_t[:,0], [100, 1700], t, f"{folder}/GT/{f(t)}.png", color=px.colors.sequential.Viridis, xrange=xrange, yrange=yrange)

    mae_max = np.abs(mae_error).numpy().max()
    # we cap the error to 15 for MAE and 300 for MSE
    plot_scatter(sub_sample_t, mae_error[idx_t,0], [-15, 15], t, f"{folder}/error/{f(t)}.png", color=px.colors.diverging.RdBu, xrange=xrange, yrange=yrange)

    mse_max = np.abs(mse_error).numpy().max()
    plot_scatter(sub_sample_t, mse_error[idx_t,0], [-300, 300], t, f"{folder}/squared_error/{f(t)}.png", color=px.colors.diverging.RdBu, xrange=xrange, yrange=yrange)


In [ ]:

import plotly.figure_factory as ff
import numpy as np
np.random.seed(1)

x = mae_error.numpy()[:,0]
hist_data = [x]
group_labels = ['distplot'] # name of the dataset

fig = ff.create_distplot(hist_data, group_labels)
fig.write_image("mae_dist.png")
fig.show()